In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
# Information about Baltimore classification of virus family groups
viral_family_groups = pd.read_csv("../../data/virus_family_groups.csv", header=0)
viral_family_groups = viral_family_groups.loc[:, ~viral_family_groups.columns.str.startswith('Unnamed')]
# Convert virus json into data frame
with open ("../../data/darkmatter/virus.json", "r") as f:
    viral_json = pd.DataFrame(json.load(f)).T
    viral_json['poly_group'] = viral_json.index.astype(np.int64)
    viral_json = viral_json.rename(columns={'samples': "full_samples"})

In [3]:
viral_json = viral_json.merge(viral_family_groups, on="family", how="left")
viral_json.loc[viral_json["family"]=="Chuviridae", "baltimore_group"] = "V"

In [4]:
viral_reads = pd.read_csv('../../data/s3/contig_quality_concat/viral_decontam.tsv', sep='\t')
poly_group_frequency = viral_reads[['poly_group', 'sample']].groupby('poly_group').count().reset_index()

In [5]:
poly_group_samples = viral_reads[['poly_group', 'sample']].groupby('poly_group')['sample'].apply(list)

In [6]:
viral_json = viral_json.merge(poly_group_frequency, on="poly_group", how="left").rename(columns = {'sample': 'n_samples'})
viral_json = viral_json.merge(poly_group_samples, on="poly_group", how="left").rename(columns = {'sample': 'samples'})

In [7]:
viral_json['n_segments'] = viral_json['segments'].apply(lambda x: len(x))

In [8]:
viral_json['sci_name'] = viral_json.apply(lambda x: x['name'] if x['name'] != 'TBD' else x['provisional_name'], 
                                              axis = 1)

In [9]:
viral_json[['novel']] = (viral_json[['name']] == 'TBD')

In [10]:
viral_json[['sci_name', 
            'novel',
            'baltimore_group',
            'genome_description',
            'number_of_segments_conventional',
            'n_segments',
            'family',
            'submission_taxid',
            'n_samples',
            'samples'
           ]].to_csv('../../data/virus_supp_table.csv', index = False)